# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [1]:
print("Hello World")

Hello World


In [1]:
import torch  # 导入 PyTorch 库

print("PyTorch 版本：", torch.__version__)  # 打印 PyTorch 的版本号

# 检查 CUDA 是否可用，并设置设备（"cuda:0" 或 "cpu"）
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("设备：", device)  # 打印当前使用的设备
print("CUDA 可用：", torch.cuda.is_available())  # 打印 CUDA 是否可用
print("cuDNN 已启用：", torch.backends.cudnn.enabled)  # 打印 cuDNN 是否已启用

# 打印 PyTorch 支持的 CUDA 和 cuDNN 版本
print("支持的 CUDA 版本：", torch.version.cuda)
print("cuDNN 版本：", torch.backends.cudnn.version())

# 创建两个随机张量（默认在 CPU 上）
x = torch.rand(5, 3)
y = torch.rand(5, 3)

# 将张量移动到指定设备（CPU 或 GPU）
x = x.to(device)
y = y.to(device)

# 对张量进行逐元素相加
z = x + y

# 打印结果
print("张量 z 的值：")
print(z)  # 输出张量 z 的内容

PyTorch 版本： 2.9.0+cu126
设备： cuda:0
CUDA 可用： True
cuDNN 已启用： True
支持的 CUDA 版本： 12.6
cuDNN 版本： 91002
张量 z 的值：
tensor([[0.8190, 1.2300, 1.8099],
        [1.3548, 0.3328, 0.7535],
        [0.5890, 0.4687, 1.0056],
        [0.9693, 1.0057, 0.4922],
        [0.6453, 1.2793, 0.7400]], device='cuda:0')
